# Imports

In [1]:
import glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import mne
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs
from mne.decoding import Scaler
from scipy import signal
from scipy.signal import butter, lfilter
from scipy.signal import ShortTimeFFT
from extraction import extract_interictal_preictal
from pipeline import Pipeline
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
import pickle

# Load in Data

In [10]:
pkl_file = os.path.join("processed_data", "final_data.pkl")
with open(pkl_file, "rb") as f:
    data = pickle.load(f)

X = data["X"]          # shape (N, 22, 1471)
y = data["y"]          # shape (N,)

print("Loaded:", X.shape, y.shape)

X = torch.tensor(X)
y = torch.tensor(y)

dataset = TensorDataset(X, y)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

Loaded: (1601, 22, 1471) (1601,)


# Creating tensors for ML

In [16]:
dataset = TensorDataset(X, y)

In [22]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

# Pickling the data

In [48]:
import pickle

# Define file names
dataset_file = "dataset.pkl"


# Save the preictal and interictal data
with open(dataset_file, "wb") as f:
    pickle.dump(dataset, f)



In [56]:

class RandomDataset(Dataset):
    def __init__(self, num_samples=100, channels=3, height=32, width=32, num_classes=10):
        self.num_samples = num_samples
        # Generate random image data (e.g., resembling 32x32 RGB images)
        self.data = tf.randn(num_samples, channels, height, width)
        # Generate random labels between 0 and num_classes-1
        self.labels = tf.randint(0, num_classes, (num_samples,))
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create the dataset and optionally inspect one batch
dataset = RandomDataset(num_samples=100)
loader = DataLoader(dataset, batch_size=10, shuffle=True)

# Print the shape of one batch as a quick test
for images, labels in loader:
    print("Batch images shape:", images.shape, "Batch labels shape:", labels.shape)
    break

# Save the dataset to disk using torch.save
dataset_filename = "random_dataset.pt"
tf.save(dataset, dataset_filename)
print(f"Dataset saved as {dataset_filename}")

Batch images shape: torch.Size([10, 3, 32, 32]) Batch labels shape: torch.Size([10])
Dataset saved as random_dataset.pt
